In [66]:
import pandas as pd
import numpy as np
import datetime 
import json

In [67]:
def is_date(strdate):
    '''判断是否是一个有效的日期字符串'''
    try:
        if ":" in strdate:
            datetime.datetime.strptime(strdate, "%Y-%m-%d %H:%M:%S")
        else:
            datetime.datetime.strptime(strdate, "%Y-%m-%d")
        return True
    except:
        return False

In [68]:
c_t = "http://www.w3.org/2002/07/owl#Class"
p_t = "http://www.w3.org/2002/07/owl#ObjectProperty"
p_t2 = "http://www.w3.org/2002/07/owl#DatatypeProperty"
label_t =  "http://www.w3.org/2000/01/rdf-schema#label"

c_xml = "http://www.openkg.cn/COVID-19/prevention/class/"
p_xml = "http://www.openkg.cn/COVID-19/prevention/property/"
r_xml = "http://www.openkg.cn/COVID-19/prevention/resource/"

r_xml2 = "http://www.openkg.cn/COVID-19/prevention#"
r_type = "http://www.w3.org/2002/07/owl#NamedIndividual"

path = r"data/防控.json"

In [69]:
'''创建Map'''
class_map = {}
property_map = {}

In [70]:
with open(path, encoding='utf-8') as f:
    f = json.load(f)
    for G in f:
        if G['@type'][0] == c_t:
            if G['@id'].find(c_xml) != -1:
                class_map[G['@id'].split(c_xml)[-1]] = G[label_t][-1]['@value']
        elif G['@type'][0] == p_t or G['@type'][0] == p_t2:
            if G['@id'].find(p_xml) != -1:
                property_map[G['@id'].split(p_xml)[-1]] = G[label_t][-1]['@value']
            else:
                property_map[G['@id'].replace(':','')] = G[label_t][-1]['@value']
        else:
            pass

In [71]:
property_map

{'http//www.openkg.cn/COVID-19/goods/property/P019': '适用人群',
 'P022': '适用场所',
 'P023': '使用交通工具',
 'P0231': '建议使用的交通工具',
 'P0232': '不建议使用的交通工具',
 'P3': '防护措施相关实体',
 'P31': '防护场所',
 'P32': '防护对象',
 'P33': '防护物资',
 'P331': '不建议使用的防护物资',
 'P332': '建议使用的防护物资',
 'P34': '防护疫情',
 'P35': '防护问答',
 'P4': '措施相关数据',
 'P41': '措施主题',
 'P42': '措施描述',
 'P5': '前提条件'}

In [72]:
entity_id = []
entity_value = []
entity_label = []
rel_from = []
rel_to = []
rel_type = []
dateC = 1
stringC = 1

entity = pd.DataFrame()
rel = pd.DataFrame()

In [73]:
with open(path, encoding='utf-8') as f:
    f = json.load(f)
    for G in f:
        if G['@type'][0] == r_type and G['@id'].find(r_xml)!=-1:
            r_id = G['@id'].split(r_xml)[-1] # main resource的 RID
            entity_id.append(r_id)
            entity_value.append(G[label_t][-1]['@value'])
            entity_label.append(class_map[G['@type'][-1].split(c_xml)[-1]])
            '''Rel'''
            for k in G.keys():
                if k.find(p_xml)!=-1:
                    # 一个PID可能有多个对应的Resource 或者是单独的String/Date
                    ktail = k.split(p_xml)[-1] #截去xml 保留P+ID
                    # 如果是string则是@value 而若是resource_map则是@id
                    if '@id' in G[k][-1]:
                        if G[k][-1]['@id'].find(r_xml)!=-1: 
                            ''' 不要map到goods或者其他表的class'''
                            for v in G[k]:
                                rel_from.append(r_id)
                                rel_type.append(property_map[ktail]) # 创建main resource和property
                                rel_to.append(v['@id'].split(r_xml)[-1])
                    elif '@value' in G[k][-1]:
                        rel_from.append(r_id)
                        rel_type.append(property_map[ktail]) # 创建main resource和property
                        
                        if is_date(G[k][-1]['@value']) == True: # 若是date字符串则 需要创建entity 并设置RID
                            dr_id = 'dR'+ str(dateC) # Character_Date dR+ID
                            dateC += 1
                            entity_id.append(dr_id)
                            entity_value.append(G[k][-1]['@value'])
                            entity_label.append('Prevention_Date')
                            rel_to.append(dr_id) # 记得加上rel_to 连接main resource
                            
                        else:
                        # 若是String的值 同样需要创建entity
                            sr_id = 'sR'+ str(stringC) # Character_String sR+ID
                            stringC += 1
                            entity_id.append(sr_id)
                            entity_value.append(G[k][-1]['@value'])
                            entity_label.append('Prevention_String')
                            rel_to.append(sr_id) # 记得加上rel_to 连接main resource

In [74]:
print(len(entity_id))
print(len(entity_value))
print(len(entity_label))
print(len(rel_from))
print(len(rel_to))
print(len(rel_type))

782
782
782
512
512
512


In [75]:
entity['Prevention:ID(Prevention-ID)'] = entity_id
entity['value'] = entity_value
entity[':LABEL'] = entity_label
rel[':START_ID(Prevention-ID)'] = rel_from
rel[':END_ID(Prevention-ID)'] = rel_to
rel[':TYPE'] = rel_type

In [76]:
entity.to_csv(r'output/prevention_entity.csv', encoding='utf-8', index=False)
rel.to_csv(r'output/prevention_rel.csv', encoding='utf-8',  index=False)